In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

import numpy as np
from pathlib import Path
import logging
from latice.index.chroma_db import (
    ChromaLatentVectorDatabaseConfig,
    ChromaLatentVectorDatabase,
)
from latice.index.raw_dp_indexer import RawDiffractionPatternIndexer, RawIndexerConfig

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

/Users/andrewtung/Documents/Github/latice


In [2]:
pattern_path = Path("data/N=100_noised.npy")
angles_path = Path("data/anglefile_N=100.txt")

batch_size = 32
random_seed = 42
image_size = (128, 128)
top_n = 10
orientation_threshold = 3.0

raw_dimension = image_size[0] * image_size[1]
print(f"Raw pattern dimension: {raw_dimension}")

Raw pattern dimension: 16384


In [3]:
chroma_raw_db_config = ChromaLatentVectorDatabaseConfig(
    persist_directory="notebook/.chroma_db_raw",
    dimension=raw_dimension,
    add_batch_size=300,
)
chroma_raw_db = ChromaLatentVectorDatabase(config=chroma_raw_db_config)

2025-04-26 23:42:03,051 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-04-26 23:42:03,136 - latice.index.chroma_db - INFO - Created persistent ChromaDB at notebook/.chroma_db_raw
2025-04-26 23:42:03,137 - latice.index.chroma_db - INFO - Retrieved existing collection 'latent_vectors'


In [4]:
raw_indexer_config = RawIndexerConfig(
    pattern_path=pattern_path,
    angles_path=angles_path,
    batch_size=batch_size,
    random_seed=random_seed,
    image_size=image_size,
    top_n=top_n,
    orientation_threshold=orientation_threshold,
)

raw_indexer = RawDiffractionPatternIndexer(
    config=raw_indexer_config,
    db=chroma_raw_db, # Pass the pre-configured DB
)

2025-04-26 23:42:03,169 - latice.index.raw_dp_indexer - INFO - Using provided ChromaLatentVectorDatabase instance.
2025-04-26 23:42:03,170 - latice.index.raw_dp_indexer - INFO - Raw index dimensionality: 16384


In [6]:
import time
from rich.progress import (
    Progress,
    SpinnerColumn,
    TimeElapsedColumn,
    BarColumn,
    TextColumn,
)

batch_patterns_np = np.load(pattern_path)
num_rows = len(batch_patterns_np)
n_samples = 1000
random_indices = np.random.choice(num_rows, size=n_samples, replace=False)
sampled_arr = batch_patterns_np[random_indices]
all_index_times = []

with Progress(
    SpinnerColumn(),
    TextColumn("[progress.description]{task.description}"),
    BarColumn(),
    TextColumn("[progress.percentage]{task.percentage:>3.0f}%"),
    TimeElapsedColumn(),
) as progress:
    task = progress.add_task("[cyan]Indexing patterns...", total=n_samples)
    for i in random_indices:
        start_time = time.time()
        orientation_results_batch = raw_indexer.index_pattern(
            batch_patterns_np[i], top_n=1, orientation_threshold=1.0
        )
        end_time = time.time()
        index_time = end_time - start_time
        all_index_times.append(index_time)
        progress.update(task, advance=1, time_elapsed=index_time)

print(
    f"Indexed {len(batch_patterns_np)} patterns in {end_time - start_time:.4f} seconds"
)
print(f"Average time per pattern: {np.mean(all_index_times):.4f} seconds")


Output()

Indexed 333227 patterns in 0.5224 seconds
Average time per pattern: 0.6104 seconds


In [10]:
mean_index_time = np.mean(all_index_times)
std_index_time = np.std(all_index_times)
print(f"Average time per pattern: {mean_index_time:.4f} seconds")
print(f"Standard deviation of time per pattern: {std_index_time:.4f} seconds")


Average time per pattern: 0.6104 seconds
Standard deviation of time per pattern: 2.8211 seconds


In [11]:
all_index_times

[89.7199969291687,
 0.6947839260101318,
 0.8829891681671143,
 0.6777560710906982,
 0.6067242622375488,
 0.7882370948791504,
 0.7300689220428467,
 0.8647809028625488,
 0.6599788665771484,
 0.6703059673309326,
 0.5409369468688965,
 0.5214283466339111,
 0.5956852436065674,
 0.7127847671508789,
 0.6707172393798828,
 0.768467903137207,
 0.6820840835571289,
 0.5663800239562988,
 0.6328082084655762,
 0.5527009963989258,
 0.4350719451904297,
 0.46880102157592773,
 0.46538591384887695,
 0.6598448753356934,
 0.6014628410339355,
 0.5862689018249512,
 0.5896761417388916,
 0.4963078498840332,
 0.6140060424804688,
 0.6030068397521973,
 0.4470357894897461,
 0.511387825012207,
 0.44502711296081543,
 0.45362305641174316,
 0.6372220516204834,
 0.6143319606781006,
 0.7181899547576904,
 0.5592939853668213,
 0.7262570858001709,
 0.568371057510376,
 0.6236190795898438,
 0.6349081993103027,
 0.6748812198638916,
 0.48334789276123047,
 0.558568000793457,
 0.4792211055755615,
 0.7296960353851318,
 0.47845315933